In [1]:
import sys, os
project_dir = os.path.abspath('..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

import datetime
from src.predictionModule.FilterSamples import FilterSamples
from src.predictionModule.LoadupSamples import LoadupSamples

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]
# Usage
logger.info("This will print to the notebook's output cell")


This will print to the notebook's output cell


In [2]:
params = {
    "daysAfterPrediction": None,
    "idxAfterPrediction": 5,
    'timesteps': 60,
    'target_option': 'mean',
    
    "LoadupSamples_tree_scaling_standard": True,
    "LoadupSamples_time_scaling_stretch": True,
    "LoadupSamples_time_inc_factor": 10,

    "FilterSamples_q_up": 0.50,

    "FilterSamples_lincomb_epochs": 5,
    "FilterSamples_lincomb_show_progress": False,
    "FilterSamples_lincomb_featureratio": 0.5,
    "FilterSamples_lincomb_itermax": 2,
    "FilterSamples_lincomb_init_toprand":  1,

    "FilterSamples_days_to_train_end": 10,
    "FilterSamples_cat_over20": True,
    "FilterSamples_cat_posOneYearReturn": False,
    "FilterSamples_cat_posFiveYearReturn": False,
    "FilterSamples_taylor_horizon_days": 20,
    "FilterSamples_taylor_roll_window_days": 20
}

In [3]:
timegroup = "group_regOHLCV_over5years"
treegroup = "group_finanTo2011"

eval_date = datetime.date(year=2025, month=6, day=13)
evaldates = [eval_date - datetime.timedelta(days=i) for i in range(1, 6)]
start_train_date = datetime.date(year=2023, month=1, day=1)
split_Date = datetime.date(year=2025, month=1, day=1)
ls = LoadupSamples(
    train_start_date=start_train_date,
    test_dates=evaldates,
    treegroup=treegroup,
    timegroup=timegroup,
    params=params,
)
ls.load_samples(main_path = "../src/featureAlchemy/bin/")
ls.split_dataset(
    start_date=start_train_date,
    last_train_date=split_Date,
    last_test_date=eval_date
)

Test date 2025-06-08 not found in the database. Omitting.
Non-finite/too-large values in train y tree: 18 samples.
Non-finite/too-large values in train X tree: 38 samples.
Non-finite/too-large values in train y time: 18 samples.
Removing 56 samples from training data.


In [4]:
train_Xtree = ls.train_Xtree
train_ytree = ls.train_ytree
train_Xtime = ls.train_Xtime
train_ytime = ls.train_ytime

test_Xtree = ls.test_Xtree
test_ytree = ls.test_ytree
test_Xtime = ls.test_Xtime
test_ytime = ls.test_ytime

treenames = ls.featureTreeNames
timenames = ls.featureTimeNames

meta_pl_train = ls.meta_pl_train
meta_pl_test = ls.meta_pl_test

In [5]:
fs = FilterSamples(
    Xtree_train=train_Xtree,
    ytree_train=train_ytree,
    treenames=treenames,
    Xtree_test=test_Xtree,
    ytree_test=test_ytree,
    meta_train=meta_pl_train,
    meta_test=meta_pl_test,
    params=params
)

In [6]:
cat_mask_train, cat_mask_test = fs.categorical_masks()

In [7]:
lincomb_mask_train, lincomb_mask_test = fs.lincomb_masks()
print(lincomb_mask_train.sum()/len(lincomb_mask_train))
print(lincomb_mask_test.sum()/len(lincomb_mask_test))

FilterSamples: (train) mean of y values 1.000985080906974.
FilterSamples: (test) mean of y values 0.9988775588421078.
FilterSamples: Starting Lincomb
  FilterSamples: Lincomb Iteration 0/2 running.
FilterSamples: Best init score 1.0182
  Mean target (train): 1.001817246273642
  Number of features selected: 262
  Max distance between idces: 21
  Mean distance between idces: 1.9965825446898002
  Median distance between idces: 1.0
  Mean target (test): 0.9993007570774955
  Fraction of days with no coverage by test mask: 0.00%
  w quantile     : 0.5314533710479736
  w_test quantile: 0.0929003581404686
    Top-0 feature: FeatureTA_volatility_bbli
    Top-1 feature: FeatureTA_trend_kst_diff
    Top-2 feature: FeatureTA_momentum_stoch_rsi
    Top-3 feature: FinData_metrics_log_trailing_pe_ratio
    Top-4 feature: MathFeature_Drawdown_MH12
    Top-5 feature: FeatureTA_momentum_pvo_hist
    Top-6 feature: MathFeature_Drawdown_MH8
    Top-7 feature: FeatureTA_trend_dpo
    Top-8 feature: Feature

In [8]:
taylor_mask_train, taylor_mask_test = fs.taylor_feature_masks()
print(taylor_mask_train.sum()/len(taylor_mask_train))
print(taylor_mask_test.sum()/len(taylor_mask_test))

FilterSamples/Taylor: (train) mean of y values 1.000985080906974.
FilterSamples/Taylor: (test) mean of y values 0.9988775588421078.
FilterSamples/Taylor: scoring features via rolling-mean Taylor approximation.
  Taylor score feat[10] FinData_quar_grossProfit_nivRev: x0=0.995902, slope=0.000000, t=6, score=0.995902
  Taylor score feat[11] FinData_quar_ebit_nivRev: x0=0.996799, slope=0.000000, t=6, score=0.996799
  Taylor score feat[12] FinData_quar_ebitda_nivRev: x0=0.997344, slope=0.000000, t=6, score=0.997344
  Taylor score feat[13] FinData_quar_totalAssets_nivRev: x0=0.998416, slope=0.000000, t=6, score=0.998416
  Taylor score feat[14] FinData_quar_totalCurrentLiabilities_nivRev: x0=0.997381, slope=0.000000, t=6, score=0.997381
  Taylor score feat[15] FinData_quar_totalShareholderEquity_nivRev: x0=0.997911, slope=0.000000, t=6, score=0.997911
  Taylor score feat[16] FinData_quar_operatingCashflow_nivRev: x0=0.997089, slope=0.000000, t=6, score=0.997089
  Taylor score feat[17] FinData